## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,59.81,96,60,6.49,broken clouds
1,1,Nouadhibou,MR,20.9310,-17.0347,71.58,64,98,8.05,haze
2,2,Jamestown,US,42.0970,-79.2353,22.71,46,100,5.99,overcast clouds
3,3,Rawson,AR,-43.3002,-65.1023,70.95,44,6,16.46,clear sky
4,4,Morant Bay,JM,17.8815,-76.4093,74.82,80,27,9.40,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#Ask customers to add min and max temps for vacation
min_temp = float(input("What is the minimum tempurature you would like for your trip?"))
max_temp = float(input("What is the maximum tempurature you would like for your trip?"))

What is the minimum tempurature you would like for your trip?45
What is the maximum tempurature you would like for your trip?75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                      (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,59.81,96,60,6.49,broken clouds
1,1,Nouadhibou,MR,20.9310,-17.0347,71.58,64,98,8.05,haze
3,3,Rawson,AR,-43.3002,-65.1023,70.95,44,6,16.46,clear sky
4,4,Morant Bay,JM,17.8815,-76.4093,74.82,80,27,9.40,scattered clouds
6,6,Half Moon Bay,US,37.4636,-122.4286,52.02,85,20,1.01,mist


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,59.81,broken clouds,-46.6000,168.3333,
1,Nouadhibou,MR,71.58,haze,20.9310,-17.0347,
3,Rawson,AR,70.95,clear sky,-43.3002,-65.1023,
4,Morant Bay,JM,74.82,scattered clouds,17.8815,-76.4093,
6,Half Moon Bay,US,52.02,mist,37.4636,-122.4286,
7,Saryagash,KZ,53.56,clear sky,41.4511,69.1701,
8,Hastings,GB,59.20,overcast clouds,50.8552,0.5729,
10,Punta Arenas,CL,46.51,light intensity shower rain,-53.1500,-70.9167,
17,Lagoa,PT,59.04,few clouds,39.0500,-27.9833,
18,Lata,PT,65.35,clear sky,40.1629,-8.3327,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?params"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
import numpy as np
# 7. Drop the rows where there is no Hotel Name.
#https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
hotel_df["Hotel Name"].replace(" ", np.nan, inplace=True)
hotel_df.dropna()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bluff,NZ,59.81,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
1,Nouadhibou,MR,71.58,haze,20.9310,-17.0347,El Medina
3,Rawson,AR,70.95,clear sky,-43.3002,-65.1023,Hosteria Sampedro
4,Morant Bay,JM,74.82,scattered clouds,17.8815,-76.4093,Whispering Bamboo Cove Resort
6,Half Moon Bay,US,52.02,mist,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
7,Saryagash,KZ,53.56,clear sky,41.4511,69.1701,fatyCentre Paradise
8,Hastings,GB,59.20,overcast clouds,50.8552,0.5729,Zanzibar Hotel
10,Punta Arenas,CL,46.51,light intensity shower rain,-53.1500,-70.9167,Hotel Hain
17,Lagoa,PT,59.04,few clouds,39.0500,-27.9833,Casa Das Faias
18,Lata,PT,65.35,clear sky,40.1629,-8.3327,Casa Maquia


In [10]:
# 8a. Create the output File (CSV)
output_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
#Heatmap for hotels with popup boxes
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.00, 31.0), zoom_level=1.3)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))